In [10]:
from IPython.display import display

In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [12]:
df = pd.read_csv("compas-scores-two-years.csv", index_col=0)

In [24]:
categorical_features = ['name','first','last','sex','race','c_charge_degree','c_charge_desc',
                        'r_charge_degree','r_charge_desc','vr_charge_degree','vr_charge_desc',
                        'type_of_assessment','v_type_of_assessment','event']

quantitative_features = ['age', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 
                         'priors_count', 'priors_count.1', 'days_b_screening_arrest', 
                         'c_days_from_compas', 'start', 'end']

ordinal_features = ['age_cat','score_text','v_score_text','decile_score','decile_score.1','v_decile_score']

X = df[categorical_features + quantitative_features].values
X_quantitative = df[quantitative_features].values
X_categorical = df[categorical_features].values
X_ordinal = df[ordinal_features].values

'name' OK 
'first' OK 
'last' OK
'compas_screening_date' : The date on which the COMPAS assessment was conducted for the person
'sex' OK
'dob' -> age (TIME STAMP)
'age' OK
'age_cat' OK
'race' OK
'juv_fel_count' OK
'decile_score' OK
'juv_misd_count' OK
'juv_other_count' OK
'priors_count' OK
'days_b_screening_arrest' OK
'c_jail_in' (TIME STAMP)
'c_jail_out' (TIME STAMP)
!!!!!! jail_duration = c_jail_out - c_jail_in ????????
'c_case_number'
'c_offense_date' (TIME STAMP)
'c_arrest_date' (TIME STAMP)
'c_days_from_compas' OK
'c_charge_degree' OK
'c_charge_desc' OK
'is_recid' 
'r_case_number'
'r_charge_degree'
'r_days_from_arrest'
'r_offense_date'
'r_charge_desc'
'r_jail_in'
'r_jail_out'
'violent_recid'
'is_violent_recid'
'vr_case_number'
'vr_charge_degree'
'vr_offense_date'
'vr_charge_desc'
'type_of_assessment'
'decile_score.1' OK
'score_text' OK
'screening_date'
'v_type_of_assessment' OK
'v_decile_score' OK
'v_score_text' OK
'v_screening_date'
'in_custody'
'out_custody'
'priors_count.1' OK
'start' OK
'end' OK
'event' OK
'two_year_recid' TARGET

## Encoding variables

### One-Hot Encoding

In [14]:
from sklearn.preprocessing import OneHotEncoder

In [15]:
one_hot_encoder = OneHotEncoder()
one_hot_encoder.fit(X_categorical)
one_hot_encoded_X = one_hot_encoder.transform(X_categorical)
one_hot_encoded_X.toarray()

array([[0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 1., 1., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.]], shape=(7214, 14806))

In [16]:
one_hot_encoder.get_feature_names_out()

array(['x0_aajah herrington', 'x0_aaliyah lovo', 'x0_aaron davis', ...,
       'x12_Risk of Violence', 'x13_0', 'x13_1'],
      shape=(14806,), dtype=object)

In [17]:
df_categorical=df[categorical_features]
categorical_cols = df.select_dtypes(include=['object']).columns
df_encoded = pd.get_dummies(df, columns=categorical_cols)

print("Number of new columns generated after one-hot encoding:", df_encoded.shape[1] - df.shape[1])

Number of new columns generated after one-hot encoding: 54953


In [18]:
X_categorical_encoded = pd.concat([df_categorical, df_encoded], axis=1)
X_categorical_encoded

,name,first,last,sex,race,c_charge_degree,c_charge_desc,r_charge_degree,r_charge_desc,vr_charge_degree,...,out_custody_2016-04-02,out_custody_2016-04-04,out_custody_2016-04-05,out_custody_2016-04-06,out_custody_2016-04-08,out_custody_2016-04-09,out_custody_2016-04-11,out_custody_2016-04-18,out_custody_2016-04-19,out_custody_2020-01-01
id,,,,,,,,,,,,,,,,,,,,,
1,miguel hernandez,miguel,hernandez,Male,Other,F,Aggravated Assault w/Firearm,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
3,kevon dixon,kevon,dixon,Male,African-American,F,Felony Battery w/Prior Convict,(F3),Felony Battery (Dom Strang),(F3),...,False,False,False,False,False,False,False,False,False,False
4,ed philo,ed,philo,Male,African-American,F,Possession of Cocaine,(M1),Driving Under The Influence,NaN,...,False,False,False,False,False,False,False,False,False,False
5,marcu brown,marcu,brown,Male,African-American,F,Possession of Cannabis,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
6,bouthy pierrelouis,bouthy,pierrelouis,Male,Other,F,arrest case no charge,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10996,steven butler,steven,butler,Male,African-American,F,Deliver Cannabis,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
10997,malcolm simmons,malcolm,simmons,Male,African-American,F,Leaving the Scene of Accident,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
10999,winston gregory,winston,gregory,Male,Other,F,Aggravated Battery / Pregnant,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False


### Ordinal Encoding

In [22]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

In [29]:
ordinal_encoder = OrdinalEncoder()
X_ordinal_encoded = np.zeros_like(X_ordinal)
import numpy as np

for i in range(X_ordinal.shape[1]):
    colonne_reshape = X_ordinal[:, i].reshape(-1, 1)
    colonne_encoded = ordinal_encoder.fit_transform(colonne_reshape)
    X_ordinal_encoded[:, i] = colonne_encoded.flatten()  # Assurez-vous de redimensionner en 1D
for i, col in enumerate(ordinal_features):
    print(f"===== Encoded classes for {col}: {ordinal_encoder.categories_}")

===== Encoded classes for age_cat: [array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype=object)]
===== Encoded classes for score_text: [array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype=object)]
===== Encoded classes for v_score_text: [array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype=object)]
===== Encoded classes for decile_score: [array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype=object)]
===== Encoded classes for decile_score.1: [array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype=object)]
===== Encoded classes for v_decile_score: [array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype=object)]


In [33]:
df_quantitative = pd.DataFrame(X_quantitative, columns=quantitative_features)
df_categorical_encoded = X_categorical_encoded
df_ordinal_encoded = pd.DataFrame(X_ordinal_encoded, columns=ordinal_features)

df_encoded = pd.concat([df_quantitative, df_categorical_encoded, df_ordinal_encoded], axis=1)

## Imputing Missing Values